# League of Data - Model Evaluation

En el Notebook anterior, **League of Data - Data Prepropocessing** se realizó todo el proceso de limipieza de la información y se generó un conjunto de datos basado en cocientes para poder comparar dos equipos y así caracterizar una partida.

En este Notebook nos enfocaremos en analizar la estructura del conjunto de datos y encontrar por medio de este, el modelo que mejor prediga cual es el equipo que resultará victorioso en una partida.

Esta meta se alcanzó cumpliendo los siguientes objetivos.

* Visualización del estado de los datos.
* Métodología de validación y Medidas de error.
* Entrenamiento de modelos.
* Analísis de características.
* Selección y extraccion de variables.

### VISUALIZACIÓN DEL ESTADO DE LOS DATOS

Ahora que se tiene el conjunto de datos a entrenar, siempre es una buena ayuda conocer como estan distribuidos los datos, como estos se ven en el espacio o en un plano. El ejercicio que se desarrolló tiene la ventaja de que es un problema de clasificación de tan solo 2 clases, sin embargo, como hemos dicho anteriormente, tiene 33 variables lo cual hace imposible visualizar toda la información sin recurrir a alguna técnica de reducción de dimensiones.

El proceso de reducción de dimensiones previo a la visualización de los datos se realizó con dos técnicas **_Principal component analysis (PCA)_** y **_t-Distributed Stochastic Neighbor Embedding (t-SNE)_**. Como la intención es poder identificar como se distribuyen los datos en el plano y en el espacio, entonces, para cada una de las técnicas se probó reduciendo las características a dos y tres.

**PCA:** Es una técnica no supervisada de extracción de características la cual tiene por objetivo conservar la mayor cantidad de información que sea posible. Lo que hace PCA es realizar una transformación de los datos originales a un nuevo conjunto de varibles, estos conjuntos se ordenan de modo que con unas pocas combinaciones se retengan la mayor cantidad de _variación_ presente en el conjunto **original** de variables.

![PCA 2D](./Images/quotients_pca-2D.png)
Como se puede observar en la gráfica, el conjunto de datos esta muy concentrado en un solo punto, tanto que pareciera que solo existe una clase, lo que nos atreveriamos a decir es que la clasificación va a ser un trabajo bastante complicado.

![PCA 3D](./Images/quotients_pca-3D.png)
Esta gráfica nos confirma aún más el estado en el que se encuentran los datos, la clase 0, correspondiente a las derrotas casi no se logra identificar.

**t-SNE:** Es una técnica no lineal de reducción de dimensiones, en términos muy vagos, **t-sne** lo que hace es formar una distribución de probabilidad sobre los datos, de tal manera que aquellos que son similares tienen una alta probabilidad de pertenecer a un mismo conjunto, mientras que los datos diferentes tienen una probabilidad extremadamente pequeña.

![t-SNE 2D](./Images/quotients_tsne-2D.png)
Con esta gráfica notamos nuevamente que ambas clases presentan un comportamiento similar, todas tienden a juntarse en el mismo lugar y la clase de las victorias intenta sobreponerse a la clase que representa las derrotas, realizar la discriminación va a ser un poco complicado.

![t-SNE 3D](./Images/quotients_tsne-3D.png)
Esta gráfica nos aporta nueva información en comparación a las 3 anteriores. La clase que representa la derrota tiene una mayor presencia, opacando casi por completo a la clase que representa las victorias, sin embargo, podemos notar que la separación de las clases sigue siendo un reto.

De las anteriores gráficas podemos concluir que es muy poco probable obtener resultados de clasificación por encima del 90%, ya que como se nota, la clases estan muy interrelacionadas, hasta el punto de no tener un solo sector en el cual se identifique a solo una de ellas.

Sin embargo, este paso se realiza para que podamos tener una idea de en que estado se encuentran los datos, los resultados definitivos se obtienen a partir del siguiente paso.

### METODOLOGÍA DE VALIDACIÓN Y MEDIDAS DE ERROR

Las métodologías de validación son aquellas estrategías que nos permiten tener una estimación sobre el comportamiento del sistema ante nuevas muestras y con base en esto determinar cual configuración de parámetros es la que mejores resultados nos entrega. En otras palabras, el mejor parámetro no es aquel que mejor se comporta con las muestras del conjunto de datos, sino aquel que mejor se comporta con las muestras desconocidas. 

Dado que cada problema de aprendizaje de máquina cuenta con un conjunto de muestras, las metodologías de validación nos permite separar las datos, de tal manera que se usa una parte para entrenar el modelo y otra para validarlo. Este proceso nos ayuda a obtener buenos resultados estadísticos sobre el desempeño del modelo.

Para efectos del problema que aquí se trata se decidió usar la validación Boostrap, con una distribución de 80% de los datos para entrenar y 20% para validar, siendo un poco más exactos, de las 5426 muestras de cocientes disponibles, 4341 se usarón para entrenar el modelo y 1085 para validarlo.

Dados estos porcentajes, el procedimiento que usa esta metodología es, seleccionar de manera aleatoria las muestras a cada uno de los conjuntos, luego se entrena y valida el modelo para después volver a seleccionar aleatoriamente conjuntos diferentes. Este procedimiento se repetirá 100 veces.

Teniendo en cuenta todo esto, para poder determinar cual modelo es el que mejor se ajusta a nuestro problema, necesitamos contar con una medida que nos indique que tan bueno es, para esto existen las medidas de error.

En un problema de clasificación como el que estamos tratando aquí, el error se da si predecimos que una muestra pertenece a una clase cuando realmente pertenece a otra, esto nos muestra que el error no tiene un grado, sino que el error existe o no existe.

Normalmente, en cualquier problema de aprendizaje de máquina se usa la eficiencia como medida de error, pero en problemas de clasificación es un arma de doble filo porque dada la definición que esta tiene, se asume que las clases poseen igual número de muestras, cuando sabemos que dicha situación no se cumple. Esto permite que la eficiencia quede sesgada a la clase con mayor número de muestras. Por esta razón, es importante tener en cuenta otro tipo de medidas de error, para este problema utilizaremos 4 medidas _sensiblidad, especificidad, eficiencia y presición_.

Para obtener los resultados de estas medidas, recurriremos a una herramienta llamada matriz de confusión, la cual esta definida de la siguiente manera.

![Matriz de confusión](./Images/cm.png)

Por medio de esta herramienta podemos definir las medidas del error. 

**Sensibilidad:** Porcentaje de positivos bien clasificados $\frac{TP}{TP+FN}$.

**Especificidad:** Porcentaje de negativos bien clasificados $\frac{TN}{TN+FP}$.

**Eficiencia:** Porcentaje de aciertos del modelo $\frac{TP+TN}{TP+TN+FP+FN}$.

**Precisión:** Porcentaje de positivos bien predichos $\frac{TP}{TP+FP}$.




### ENTRENAMIENTO DE LOS MODELOS

Después de haber analizado un poco la distribución de los datos, nos dimos cuenta que no se logra percibir un patrón de comportamiento en ellos, lo cual, además de complicarnos la discriminación de las clases, también nos pone difícil la tarea de seleccionar el modelo más pertinente. 

Dado lo anterior se decidió analizar el problema con 8 modelos distintos (Regresión Lineal, Ventana de Parzen, KNN, Random Forest, XGBoost, Redes Neuronales, Redes Neuronales Profundas, Máquinas de Soporte Vectorial), cada uno de ellos tiene una serie de hiperparámetros que dependiendo de su configuración presentan resultados diferentes. Por lo tanto, se ha establecido una malla de valores en cada modelo. 

Para cada configuración de hiperparámetros en su respectivo modelo, se realizó la misma metodología de validación con iguales condiciones, esto con el fin tener un punto en común de evaluación.

Dado que algunos modelos cuentan con una cantidad considerable de configuraciones para sus hiperparámetros, los resultados que se expondrán en este Notebook serán aquellos que presenten el mejor resultado. Estos resultados se mostrarán por medio de dos gráficas,  La Curva ROC y el consolidado de las medidas de error.

* **Regresión Linear**

Es considerado el modelo más elemental dentro del aprendizaje de máquina, sin embargo, se ha demostrado que bajo ciertos escenarios presenta unos grandes resultados. 

El hiperparámetro que se tuvo en cuenta para este modelo es el grado del polinomio y los valores con que se probó fueron: _polinomio grado 1 y 2_, siendo el polinomio de grado 1 la mejor configuración para este modelo.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![logistic_ROC](./Images/logistic_ROC.png) | ![logistic_Measures](./Images/logistic_Measures.png) |


* **k-Nearest Neighbors (KNN)**

Este es un modelo que evalua todas las muestras que contiene el conjunto de datos y con base en alguna medida de similaridad predice para un valor para una nueva muestra. Para los problemas de clasificación como este, se evalua cuantas muestras son las más cercanas a la nueva y por medio de la _moda_ se identifica a cual clase pertenece.

Los hiperparámetros con los que cuenta este modelo son el número de vecinos cercanos y la medida de distancia. Para el primero se tomó en cuenta los valores 1,2,3,4,5,10 y 100; en cuanto al segundo, solo se tomó en cuenta una configuración y fue la distancia euclidiana.

El mejor resultado se obtuvo con la configuración de 100 vecinos.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![knn_ROC](./Images/knn_ROC.png) | ![knn_Measures](./Images/knn_Measures.png) |


* **Ventana de Parzen**

Este modelo es muy similar al anteriormente mencionado (KNN), la diferencia es que en este modelo no se da la definición de "_vecindad_" (Área variable definida por los puntos más cercanos), sino que por medio de una función Kernel genera una función de densidad de probabilidad. 

El hiperparámetro que se configura en este modelo es el ancho de la ventana, en otras palabras se modifica el peso que se le asigna a las muestras, entre más cerca, más peso asigna y mientras más lejos menos peso. La malla de valores asignada fue 0.05, 0.1, 0.5, 1, 10.

El mejor resultado se dío con la ventana de ancho 0.5.

![parzenwindow_Measures](./Images/parzenwindow_Measures.png)

